In [57]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime, date
import sqlite3
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import pandas as pd
import logging
import threading
import matplotlib.pyplot as plt

In [59]:
# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def init_db():
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS people 
                 (id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT NOT NULL, photo_path TEXT NOT NULL)''')
    c.execute('''CREATE TABLE IF NOT EXISTS attendance 
                 (id INTEGER, name TEXT, timestamp TEXT, FOREIGN KEY(id) REFERENCES people(id))''')
    conn.commit()
    conn.close()
    logging.info("Database initialized.")

In [61]:
def load_known_faces():
    known_faces = {}
    conn = sqlite3.connect('attendance.db')
    c = conn.cursor()
    c.execute("SELECT name, photo_path FROM people")
    rows = c.fetchall()
    if not rows:
        logging.warning("No people found in the database.")
    for name, photo_path in rows:
        try:
            image = face_recognition.load_image_file(photo_path)
            encoding = face_recognition.face_encodings(image)
            if encoding:
                known_faces[name] = [encoding[0]]
                logging.info(f"Loaded face encoding for {name}")
            else:
                logging.warning(f"No face encoding found in {photo_path} for {name}")
        except Exception as e:
            logging.error(f"Error loading face for {name}: {e}")
    conn.close()
    return known_faces

In [63]:
class AddPersonGUI:
    def __init__(self, root, status_callback):
        self.root = root
        self.root.title("Add New Person")
        self.root.geometry("640x650")
        self.root.configure(bg="#e6f0fa")
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.status_callback = status_callback

        self.frame = ttk.Frame(self.root, padding="20", style="My.TFrame")
        self.frame.pack(fill='both', expand=True)

        ttk.Label(self.frame, text="Add New Person", font=("Segoe UI", 18, "bold"), foreground="#1e90ff").pack(pady=10)

        ttk.Label(self.frame, text="Enter Name:", font=("Segoe UI", 12)).pack(pady=5)
        self.name_entry = ttk.Entry(self.frame, width=30, font=("Segoe UI", 12))
        self.name_entry.pack(pady=5)

        self.capture_btn = ttk.Button(self.frame, text="📸 Capture Photo", command=self.capture_photo, style="Accent.TButton")
        self.capture_btn.pack(pady=10)
        self.capture_btn.config(state='disabled')

        self.submit_btn = ttk.Button(self.frame, text="✔ Submit", command=self.submit_person, style="Accent.TButton")
        self.submit_btn.pack(pady=10)
        self.submit_btn.config(state='disabled')

        self.video_label = ttk.Label(self.frame, text="Loading camera...", font=("Segoe UI", 12), foreground="gray")
        self.video_label.pack(pady=10, expand=True)

        self.status_var = tk.StringVar(value="Initializing camera...")
        ttk.Label(self.frame, textvariable=self.status_var, font=("Segoe UI", 10), foreground="gray").pack(pady=10)

        self.cap = None
        self.camera_ready = False
        threading.Thread(target=self.init_camera, daemon=True).start()

    def init_camera(self):
        try:
            self.cap = cv2.VideoCapture(0)
            if not self.cap.isOpened():
                self.root.after(0, lambda: messagebox.showerror("Error", "Camera not accessible! Please check your webcam."))
                self.root.after(0, self.on_closing)
                return
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
            self.camera_ready = True
            self.root.after(0, self.enable_buttons)
            self.root.after(0, lambda: self.status_var.set("Camera ready"))
            self.root.after(0, lambda: self.status_callback("Camera initialized"))
            self.show_video()
        except Exception as e:
            logging.error(f"Error initializing camera: {e}")
            self.root.after(0, lambda: messagebox.showerror("Error", f"Camera initialization failed: {e}"))
            self.root.after(0, self.on_closing)

    def enable_buttons(self):
        self.capture_btn.config(state='normal')
        self.submit_btn.config(state='normal')
        self.video_label.config(text="")

    def show_video(self):
        if self.camera_ready and self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                frame = cv2.resize(frame, (640, 480), interpolation=cv2.INTER_AREA)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame)
                imgtk = ImageTk.PhotoImage(image=img)
                self.video_label.imgtk = imgtk
                self.video_label.configure(image=imgtk)
            self.root.after(10, self.show_video)

    def capture_photo(self):
        if self.camera_ready and self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                self.photo = frame
                cv2.imwrite("temp_photo.jpg", frame)
                messagebox.showinfo("Success", "Photo captured!")
                self.status_var.set("Photo captured")
                self.status_callback("Photo captured")
                logging.info("Photo captured successfully.")
            else:
                messagebox.showerror("Error", "Failed to capture photo!")
                self.status_var.set("Capture failed")
                self.status_callback("Capture failed")
                logging.error("Failed to capture photo.")

    def submit_person(self):
        name = self.name_entry.get().strip()
        if not name or not hasattr(self, 'photo'):
            messagebox.showerror("Error", "Please enter a name and capture a photo!")
            self.status_var.set("Error: Missing name or photo")
            self.status_callback("Error: Missing name or photo")
            return

        photo_path = f"dataset/{name}.jpg"
        os.makedirs("dataset", exist_ok=True)
        cv2.imwrite(photo_path, self.photo)

        conn = sqlite3.connect('attendance.db')
        c = conn.cursor()
        c.execute("INSERT INTO people (name, photo_path) VALUES (?, ?)", (name, photo_path))
        conn.commit()
        conn.close()

        messagebox.showinfo("Success", f"{name} added successfully!")
        self.status_var.set(f"{name} added")
        self.status_callback(f"{name} added to database")
        logging.info(f"{name} added to database.")
        self.on_closing()

    def on_closing(self):
        if hasattr(self, 'cap') and self.cap and self.cap.isOpened():
            self.cap.release()
        self.root.destroy()
        logging.info("Add Person GUI closed.")

In [129]:
def mark_attendance(status_callback):
    logging.info("Starting attendance system...")
    known_faces = load_known_faces()
    if not known_faces:
        messagebox.showwarning("Warning", "No known faces in the database. Please add people first.")
        status_callback("No known faces in database")
        logging.warning("No known faces loaded.")
        return

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror("Error", "Camera not accessible! Please check your webcam.")
        status_callback("Camera not accessible")
        logging.error("Camera not accessible.")
        return

    resolutions = [(640, 480), (320, 240), (800, 600)]
    success = False
    for width, height in resolutions:
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
        if cap.isOpened():
            success = True
            logging.info(f"Camera set to resolution: {width}x{height}")
            break
    if not success:
        messagebox.showerror("Error", "Failed to set a valid camera resolution.")
        cap.release()
        status_callback("Failed to set camera resolution")
        logging.error("Failed to set a valid camera resolution.")
        return

    status_callback("Attendance camera ready")

    # Create a Tkinter window for the attendance display
    attendance_window = tk.Toplevel()
    attendance_window.title("Attendance System")
    attendance_window.geometry("650x500")
    attendance_window.configure(bg="#e6f0fa")
    attendance_window.protocol("WM_DELETE_WINDOW", lambda: on_attendance_close(attendance_window, cap, status_callback))

    # Embed matplotlib figure in Tkinter
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
    fig, ax = plt.subplots(figsize=(6, 4))
    canvas = FigureCanvasTkAgg(fig, master=attendance_window)
    canvas.draw()
    canvas.get_tk_widget().pack(pady=10, expand=True)

    stop_button = ttk.Button(attendance_window, text="Stop Attendance", command=lambda: on_attendance_close(attendance_window, cap, status_callback), style="Accent.TButton")
    stop_button.pack(pady=10)

    try:
        today = date.today().strftime("%Y-%m-%d")
        logging.info(f"Today’s date: {today}")
        attended_today = set()  # Set to track individuals who attended today

        def update_frame(cap_ref):
            nonlocal cap, attended_today  # Allow modification of the outer cap and attended_today
            if attendance_window.winfo_exists():  
                ret, frame = cap_ref.read()
                if not ret:
                    logging.error("Failed to capture frame from camera. Attempting to reinitialize...")
                    status_callback("Failed to capture frame, reinitializing...")
                    cap_ref.release()
                    cap_ref = cv2.VideoCapture(0)
                    if not cap_ref.isOpened():
                        logging.error("Reinitialization failed. Stopping attendance.")
                        attendance_window.after(100, lambda: on_attendance_close(attendance_window, cap_ref, status_callback))
                        return
                    cap = cap_ref  # Update the outer cap reference
                    for width, height in resolutions:
                        cap_ref.set(cv2.CAP_PROP_FRAME_WIDTH, width)
                        cap_ref.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
                        if cap_ref.isOpened():
                            logging.info(f"Reinitialized camera to resolution: {width}x{height}")
                            break
                    status_callback("Camera reinitialized")
                    logging.info("Camera reinitialized successfully.")
                    attendance_window.after(10, lambda: update_frame(cap_ref))  # Retry
                    return

                # Log frame details for debugging
                logging.debug(f"Captured frame shape: {frame.shape if frame is not None else 'None'}")
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) if ret else np.zeros((480, 640, 3), dtype=np.uint8)
                face_locations = face_recognition.face_locations(rgb_frame)
                face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

                for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                    name = "Unknown"
                    message = ""
                    for person_name, encodings in known_faces.items():
                        matches = face_recognition.compare_faces(encodings, face_encoding, tolerance=0.6)
                        if True in matches:
                            name = person_name
                            break

                    cv2.rectangle(rgb_frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    cv2.putText(rgb_frame, name, (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                    if name != "Unknown" and name not in attended_today:
                        conn = sqlite3.connect('attendance.db')
                        c = conn.cursor()
                        c.execute("SELECT timestamp FROM attendance WHERE name = ? AND timestamp LIKE ?",
                                  (name, f"{today}%"))
                        existing_record = c.fetchone()

                        if existing_record:
                            message = "Attendance already taken"
                            logging.info(f"{name}: Attendance already taken at {existing_record[0]}")
                            status_callback(f"{name}: Already marked today")
                            attended_today.add(name)  # Mark as attended after first check
                        else:
                            c.execute("SELECT id FROM people WHERE name = ?", (name,))
                            person_id = c.fetchone()
                            if person_id:
                                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                c.execute("INSERT INTO attendance (id, name, timestamp) VALUES (?, ?, ?)",
                                          (person_id[0], name, timestamp))
                                conn.commit()
                                message = "Attendance taken"
                                logging.info(f"Attendance marked for {name} at {timestamp}")
                                status_callback(f"{name}: Attendance marked")
                                attended_today.add(name)  # Mark as attended after marking
                            else:
                                logging.error(f"No ID found for {name} in people table.")
                        conn.close()
                    elif name in attended_today:
                        message = "Attendance already marked today"

                    if message:
                        cv2.putText(rgb_frame, message, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
                                    (0, 255, 0) if "taken" in message else (0, 0, 255), 2)

                # Update matplotlib display
                ax.clear()
                ax.imshow(rgb_frame)
                ax.set_title("Attendance System")
                canvas.draw()

                # Schedule the next frame update
                attendance_window.after(10, lambda: update_frame(cap_ref))

        update_frame(cap)  # Start the frame update loop with the initial cap
    except Exception as e:
        logging.error(f"Error in attendance system: {e}")
        status_callback(f"Error: {e}")
    finally:
        if 'cap' in locals() and cap.isOpened():
            cap.release()
        plt.ioff()
        plt.close(fig)
        logging.info("Attendance system resources cleaned up.")

In [131]:
def on_attendance_close(window, cap, status_callback):
    if cap.isOpened():
        cap.release()
    window.destroy()
    status_callback("Attendance system stopped")
    logging.info("Attendance system stopped by user.")

In [133]:
def view_attendance(status_callback):
    conn = sqlite3.connect('attendance.db')
    df = pd.read_sql_query("SELECT * FROM attendance", conn)
    conn.close()

    if df.empty:
        messagebox.showinfo("Attendance", "No attendance records found!")
        status_callback("No attendance records found")
        return

    top = tk.Toplevel()
    top.title("Attendance Records")
    top.geometry("500x400")
    top.configure(bg="#e6f0fa")
    top.protocol("WM_DELETE_WINDOW", lambda: top.destroy())

    frame = ttk.Frame(top, padding="20", style="My.TFrame")
    frame.pack(fill='both', expand=True)

    ttk.Label(frame, text="Attendance Records", font=("Segoe UI", 18, "bold"), foreground="#1e90ff").pack(pady=10)

    text = tk.Text(frame, height=15, width=60, font=("Segoe UI", 10))
    text.pack(expand=True, fill='both')
    text.insert(tk.END, df.to_string(index=False))
    text.config(state='disabled')

    status_callback("Viewing attendance records")

In [135]:
def main():
    init_db()
    root = tk.Tk()
    root.title("Face Recognition Attendance System")
    root.geometry("400x350")
    root.configure(bg="#e6f0fa")
    root.protocol("WM_DELETE_WINDOW", lambda: on_closing(root))
    root.resizable(True, True)

    style = ttk.Style()
    style.theme_use('clam')
    style.configure("My.TFrame", background="#e6f0fa")
    style.configure("TButton", font=("Segoe UI", 12), padding=10)
    style.configure("Accent.TButton", background="#1e90ff", foreground="white")
    style.map("Accent.TButton", background=[('active', '#4169e1')])

    canvas = tk.Canvas(root, bg="#e6f0fa")
    scrollbar = ttk.Scrollbar(root, orient="vertical", command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas, style="My.TFrame")

    scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    def on_mouse_wheel(event):
        canvas.yview_scroll(-1 * (event.delta // 120), "units")

    root.bind_all("<MouseWheel>", on_mouse_wheel)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")
    
    ttk.Label(scrollable_frame, text="Attendance System", font=("Segoe UI", 20, "bold"), foreground="#1e90ff").pack(pady=20)

    ttk.Button(scrollable_frame, text="➕ Add New Person", 
               command=lambda: AddPersonGUI(tk.Toplevel(root), lambda msg: status_var.set(msg)), 
               style="Accent.TButton").pack(pady=10, fill='x', padx=20)
    ttk.Button(scrollable_frame, text="▶ Start Attendance", 
               command=lambda: mark_attendance(lambda msg: status_var.set(msg)), 
               style="Accent.TButton").pack(pady=10, fill='x', padx=20)
    ttk.Button(scrollable_frame, text="📋 View Attendance", 
               command=lambda: view_attendance(lambda msg: status_var.set(msg)), 
               style="Accent.TButton").pack(pady=10, fill='x', padx=20)
    ttk.Button(scrollable_frame, text="✖ Exit", 
               command=lambda: on_closing(root), 
               style="Accent.TButton").pack(pady=10, fill='x', padx=20)

    status_var = tk.StringVar(value="System Ready")
    ttk.Label(scrollable_frame, textvariable=status_var, font=("Segoe UI", 10), foreground="gray").pack(pady=10)

    root.mainloop()

In [143]:
def on_closing(root):
    logging.info("Closing application...")
    if 'cap' in globals() and cap and cap.isOpened():
        cap.release()
    root.quit()
    root.destroy()
    logging.info("Application closed successfully.")

if __name__ == "__main__":
    main()

2025-07-02 04:30:28,767 - INFO - Database initialized.
2025-07-02 04:31:18,057 - INFO - Photo captured successfully.
2025-07-02 04:31:22,060 - INFO - John Doe added to database.
2025-07-02 04:31:22,333 - INFO - Add Person GUI closed.
2025-07-02 04:31:23,375 - INFO - Starting attendance system...
2025-07-02 04:31:23,981 - INFO - Loaded face encoding for John Doe
2025-07-02 04:31:24,601 - INFO - Camera set to resolution: 640x480
2025-07-02 04:31:24,659 - INFO - Today’s date: 2025-07-02
2025-07-02 04:31:25,326 - DEBUG - Captured frame shape: (480, 640, 3)
2025-07-02 04:31:25,936 - INFO - Attendance marked for John Doe at 2025-07-02 04:31:25
2025-07-02 04:31:26,258 - INFO - Attendance system resources cleaned up.
2025-07-02 04:31:26,258 - ERROR - Failed to capture frame from camera. Attempting to reinitialize...
2025-07-02 04:31:26,897 - INFO - Reinitialized camera to resolution: 640x480
2025-07-02 04:31:26,898 - INFO - Camera reinitialized successfully.
2025-07-02 04:31:27,590 - DEBUG - C